In [1]:
%matplotlib qt
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import dataloader
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv('./dataset-daily.csv')
dataset.head()

,time,air_temperature_mean,pressure,wind_direction,wind_speed
0,19950101,2.7,990.6667,263,6.7
1,19950102,0.6,1000.7083,301,4.1
2,19950103,-0.3,1027.3750,355,3.2
3,19950104,-2.8,1035.9584,86,1.6
4,19950105,-5.1,1034.5416,124,3.5


In [3]:
trainset, testset = train_test_split(dataset, test_size = 0.1)
trainset, valset = train_test_split(trainset, test_size = 0.1)

In [4]:
trainset.iloc[0]

time                    2.004012e+07
air_temperature_mean   -6.400000e+00
pressure                1.023375e+03
wind_direction          1.410000e+02
wind_speed              2.800000e+00
Name: 3309, dtype: float64

In [5]:
class WindSpeedDataset(Dataset):
    def __init__(self, dataframe, suffle=False, transform=None):
        self.dataframe = dataframe.copy()
        dataframe1 = dataframe.copy()
        self.suffle = suffle
        self.transform = transform
        
        if 'time' in dataframe1:
            dataframe1.pop('time')
        if 'wind_speed' in dataframe1:
            self.labelset = dataframe1.pop('wind_speed')
            
        self.featureset = dataframe1
        
    def __len__(self):
        return len(self.featureset)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        # print(idx)
        
        label = np.array([self.labelset.iloc[idx]])
        features = self.featureset.iloc[idx].to_numpy()
        
        sample =(features, label)
        
        if self.transform:
            sample = self.transform(sample)

        return sample

In [6]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        data, label = sample
        return (torch.from_numpy(data),torch.from_numpy(label))

In [7]:
class ComposeTransform(object):
    """Composes several transforms together.

    Args:
        transforms (list of ``Transform`` objects): list of transforms to compose.

    Example:
        >>> ComposeTransform([
        >>>     ToTensor(),
        >>> ])
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, data):
        for t in self.transforms:
            img = t(data)
        return img

    def __repr__(self):
        format_string = self.__class__.__name__ + '('
        for t in self.transforms:
            format_string += '\n'
            format_string += '    {0}'.format(t)
        format_string += '\n)'
        return format_string

In [8]:
train_dataset = WindSpeedDataset(trainset,transform=ComposeTransform([ToTensor()]))
test_dataset = WindSpeedDataset(testset, transform=ComposeTransform([ToTensor()]))
val_dataset = WindSpeedDataset(valset, transform=ComposeTransform([ToTensor()]))

In [9]:
train_dataset[0]

(tensor([  -6.4000, 1023.3750,  141.0000], dtype=torch.float64),
 tensor([2.8000], dtype=torch.float64))

In [10]:
batch_size = 5

trainloader = dataloader.DataLoader(train_dataset, batch_size, shuffle = True)
valloader = dataloader.DataLoader(val_dataset, batch_size, shuffle = False)
testloader = dataloader.DataLoader(test_dataset, batch_size, shuffle = False)

In [11]:
f, l = next(iter(trainloader))

In [12]:
f

tensor([[ 1.7500e+01,  1.0093e+03,  2.7200e+02],
        [ 1.4300e+01,  1.0203e+03,  3.0700e+02],
        [ 1.6500e+01,  1.0132e+03,  2.5900e+02],
        [ 2.7000e+00,  9.9842e+02,  1.4400e+02],
        [-6.0000e-01,  1.0070e+03,  2.8800e+02]], dtype=torch.float64)

In [13]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 30)
        self.fc2 = nn.Linear(30, 1)
        
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [14]:
model = Net()
print(model)

Net(
  (fc1): Linear(in_features=3, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=1, bias=True)
)


In [15]:
from torch.optim import SGD
criterion = nn.MSELoss()
optimizer = SGD(params=model.parameters(), lr=0.01)

In [16]:
trainlosses = []
testlosses = []
epochs = 5


for epoch in range(epochs):
    trainloss = 0
    model.train()
    for batch, (data, target) in enumerate(trainloader):
        data = data.type(torch.FloatTensor)
        target = target.type(torch.FloatTensor)
#         if cuda:
#             image, target = image.cuda(), target.cuda()
        
        optimizer.zero_grad()
        
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        trainloss += loss.item()
        
        if batch % 20 == 0:
            testloss = 0
            model.eval()
            with torch.no_grad():
                for data, target in valloader:
                    data = data.type(torch.FloatTensor)
                    target = target.type(torch.FloatTensor)
#                     if cuda:
#                         image, target = image.cuda(), target.cuda()
                    ps = model(data)
                    testloss += criterion(ps, target).item()
                testloss = testloss / len(valloader)        
            trainloss = trainloss / len(trainloader)
        
            trainlosses.append(trainloss)
            testlosses.append(testloss)
        
            print(f'Epoch: {epoch}',
                  f'Batch: {batch} out of {len(trainloader)}',
                  f'Training Loss: {trainloss}',
                  f'Test Loss: {testloss}')

Epoch: 0 Batch: 0 out of 592Training Loss: 39.43460990287162 Test Loss: 1.7769277552002376e+18
Epoch: 0 Batch: 20 out of 592Training Loss: 2914969709586819.5 Test Loss: 318577857511920.5
Epoch: 0 Batch: 40 out of 592Training Loss: 12456507950623.791 Test Loss: 141990303319505.47
Epoch: 0 Batch: 60 out of 592Training Loss: 3378312826246.108 Test Loss: 63285166540737.94
Epoch: 0 Batch: 80 out of 592Training Loss: 1502044077061.8481 Test Loss: 28206211165339.152
Epoch: 0 Batch: 100 out of 592Training Loss: 669455247053.8478 Test Loss: 12571523892689.455
Epoch: 0 Batch: 120 out of 592Training Loss: 298376540757.96936 Test Loss: 5603132678888.728
Epoch: 0 Batch: 140 out of 592Training Loss: 132986518743.49657 Test Loss: 2497318366859.636
Epoch: 0 Batch: 160 out of 592Training Loss: 59272144092.03969 Test Loss: 1113055525919.0303
Epoch: 0 Batch: 180 out of 592Training Loss: 26417611432.15547 Test Loss: 496089242903.2727
Epoch: 0 Batch: 200 out of 592Training Loss: 11774335392.284046 Test Los

Epoch: 2 Batch: 580 out of 592Training Loss: 0.06460335542146171 Test Loss: 2.12873207664851
Epoch: 3 Batch: 0 out of 592Training Loss: 0.002331554688311912 Test Loss: 2.134601447279706
Epoch: 3 Batch: 20 out of 592Training Loss: 0.08737311230283952 Test Loss: 2.128901807873538
Epoch: 3 Batch: 40 out of 592Training Loss: 0.07689909995183117 Test Loss: 2.1270535441510603
Epoch: 3 Batch: 60 out of 592Training Loss: 0.07865896768768267 Test Loss: 2.1262907148762182
Epoch: 3 Batch: 80 out of 592Training Loss: 0.0681860285324053 Test Loss: 2.125666715656266
Epoch: 3 Batch: 100 out of 592Training Loss: 0.06043144806100613 Test Loss: 2.1272471070741164
Epoch: 3 Batch: 120 out of 592Training Loss: 0.0886548966931846 Test Loss: 2.129651915620674
Epoch: 3 Batch: 140 out of 592Training Loss: 0.05449598034403606 Test Loss: 2.1254517898866623
Epoch: 3 Batch: 160 out of 592Training Loss: 0.06601130457276465 Test Loss: 2.1259774924679236
Epoch: 3 Batch: 180 out of 592Training Loss: 0.0759896377670663

In [19]:
plt.plot(testlosses)